In [1]:
from keras import applications, optimizers
from keras import backend as K
from keras import models as Models
from keras.models import Model 
from keras.layers import Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator

import numpy as np
import os
import sys
import time

img_width, img_height = 256, 256
train_data_dir = "train_set"
validation_data_dir = "test_set"
batch_size = 16
epochs = 5

Using TensorFlow backend.


In [2]:
model = applications.VGG16(weights = "vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5", include_top=False, input_shape = (img_width, img_height, 3))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [3]:
# Create output
x = model.output
x = Flatten()(x)
predictions = Dense(10, activation="softmax")(x)

# creating the final model 
model_final = Model(inputs = model.input, outputs = predictions)

# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

print(model_final.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [4]:
# Initiate the train and test generators with data Augumentation 
train_datagen_rescale = ImageDataGenerator(rescale = 1./255)

train_datagen_aug = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

test_datagen_rescale = ImageDataGenerator(rescale = 1./255)

test_datagen_aug = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

train_generator = train_datagen_aug.flow_from_directory(
train_data_dir,
target_size = (img_height, img_width),
batch_size = batch_size, 
class_mode = "categorical")

validation_generator = test_datagen_aug.flow_from_directory(
validation_data_dir,
target_size = (img_height, img_width),
batch_size = batch_size, 
class_mode = "categorical")

Found 1450 images belonging to 10 classes.
Found 264 images belonging to 10 classes.


In [5]:
# Train the model
time_start = time.time()

model_final.fit_generator(
    train_generator,
    epochs = epochs,
    validation_data = validation_generator)

time_elapsed = time.time() - time_start

# Record time used
print(time_elapsed)

Epoch 1/5
91/91 [==============================] - 65s 717ms/step - loss: 1.4510 - acc: 0.5431 - val_loss: 1.0274 - val_acc: 0.6970
Epoch 2/5
91/91 [==============================] - 58s 637ms/step - loss: 0.8667 - acc: 0.7392 - val_loss: 0.6930 - val_acc: 0.7765
Epoch 3/5
91/91 [==============================] - 58s 641ms/step - loss: 0.6540 - acc: 0.7946 - val_loss: 0.5361 - val_acc: 0.8258
Epoch 4/5
91/91 [==============================] - 58s 637ms/step - loss: 0.5822 - acc: 0.8095 - val_loss: 0.4539 - val_acc: 0.8409
Epoch 5/5
91/91 [==============================] - 60s 663ms/step - loss: 0.4547 - acc: 0.8552 - val_loss: 0.4133 - val_acc: 0.8674
300.5879456996918


In [7]:
score = model_final.evaluate_generator(validation_generator)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 0.365571645794
Test accuracy: 0.878787878788
